In [1]:
import pandas as pd
import pickle
import numpy as np
import tqdm
import pandas as pd
import json

get_all_games = 0
get_game_order = 1
start_val = 500

In [2]:
if get_all_games:
    query = """
    select * from `analytics-156605.barath.ludo_scores`
    """

    all_games = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    all_games.sort_values(by='time_stamp')
    
    all_games = all_games.to_dict('records')

    with open('../data/all_game_score_aug_16_to_sept_15.pkl', 'wb') as f:
        pickle.dump(all_games, f)

else:

    with open('../data/all_game_score_aug_16_to_sept_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

all_games = {item['division']:item for item in all_games if item is not None}

In [3]:
if get_game_order:
    query = """
    select distinct game_ref_id, date from (
    select distinct game_ref_id, updated_timestamp, dt as date
    from `analytics-156605.rush_app_bi.transactions_ledger_extended_full`
    where transaction_sub_category = 'TotalFee' and transaction_category = 'GamePlay' and platform = 'cashApp' and transaction_status = 'SUCCESS' and dt between "2022-08-16" and "2022-09-15" and game_ref_id like 'LUDO%'
    order by updated_timestamp)
    """

    game_order = pd.read_gbq(query, use_bqstorage_api=True)

    game_order = game_order.to_dict('records')

    with open('../data/all_games_aug_16_to_sept_15.pkl', 'wb') as f:
        pickle.dump(game_order, f)

else:

    with open('../data/all_games_aug_16_to_sept_15.pkl', 'rb') as f:
        game_order = pickle.load(f)

In [4]:
for i in tqdm.tqdm(all_games):
    all_games[i]['score_1'] = np.sum(json.loads(all_games[i]['score_1']))
    all_games[i]['score_2'] = np.sum(json.loads(all_games[i]['score_2']))

100%|██████████| 14852668/14852668 [04:54<00:00, 50490.13it/s]


In [5]:
live_scores = {}

for d in tqdm.tqdm(game_order):
    if d['game_ref_id'] in all_games:
        data = all_games[d['game_ref_id']]

        if data['from_user'] not in live_scores:
            live_scores[data['from_user']] = start_val
        if data['opponent'] not in live_scores:
            live_scores[data['opponent']] = start_val

        all_games[d['game_ref_id']]['new_skill_1'] = live_scores[data['from_user']]
        all_games[d['game_ref_id']]['new_skill_2'] = live_scores[data['opponent']]

        live_scores[data['from_user']] += (data['score_1'] - data['score_2'])
        live_scores[data['opponent']] += (data['score_2'] - data['score_1'])

100%|██████████| 19642895/19642895 [01:26<00:00, 227118.19it/s]


In [6]:
win = []
q_pred = []
skill_pred = []
q_score_0 = []
q_score_1 = []

X = []
X_test = []
y = []
y_test = []

for i in tqdm.tqdm(all_games):
    if 'new_skill_1' in all_games[i]:
        data = all_games[i]

        if str(data['dt']) in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:
            win.append(data['score_1'] > data['score_2'])
            q_pred.append(data['new_skill_1'] > data['new_skill_2'])
            skill_pred.append((data['mu'] - 3*data['sigma']) > (data['mu_opp'] - 3*data['sigma_opp']))

            q_score_0.append(data['new_skill_1'])
            q_score_1.append(data['new_skill_2'])

        if str(data['dt']) not in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:
            X.append([data['mu'], data['sigma'], data['new_skill_1'], 
                            data['mu_opp'], data['sigma_opp'], data['new_skill_2']])
            y.append(data['score_1'] > data['score_2'])
        else:
            X_test.append([data['mu'], data['sigma'], data['new_skill_1'], 
                            data['mu_opp'], data['sigma_opp'], data['new_skill_2']])
            y_test.append(data['score_1'] > data['score_2'])

100%|██████████| 14852668/14852668 [01:30<00:00, 163310.42it/s]


In [7]:
win = np.array(win).astype('int32')
q_pred = np.array(q_pred).astype('int32')
skill_pred = np.array(skill_pred).astype('int32')

q_score_0 = np.array(q_score_0)
q_score_1 = np.array(q_score_1)

bins = np.abs(q_score_0 - q_score_1)

X = np.array(X)
X_test = np.array(X_test)
y = np.array(y)
y_test = np.array(y_test)

In [8]:
u1 = []
tot = []
for i in range(50):
    idx = np.logical_and(bins>i*1000, bins<(i+1)*1000)
    temp = win[idx]
    u1.append(np.sum(temp[q_score_0[idx]>q_score_1[idx]]) + np.sum(temp[q_score_0[idx]<q_score_1[idx]]==0))
    tot.append(len(q_score_0[idx]))

In [9]:
import pandas as pd
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,519558,1030977
1,276461,538569
2,159025,305591
3,98032,186364
4,63303,119587
5,43025,80376
6,29734,55441
7,21512,39764
8,15775,28811
9,11482,21113


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
clf = Pipeline([('scaler', StandardScaler()), ('gbc', GradientBoostingClassifier(random_state=42, verbose=1))])
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(X, y)

      Iter       Train Loss   Remaining Time 
         1           1.3702           70.96m
         2           1.3699           70.20m
         3           1.3696           69.33m
         4           1.3693           68.36m
         5           1.3690           67.59m
         6           1.3688           66.89m
         7           1.3686           66.20m
         8           1.3684           65.60m
         9           1.3682           65.00m
        10           1.3680           64.39m
        20           1.3668           57.39m
        30           1.3661           50.30m
        40           1.3654           43.10m
        50           1.3650           35.91m
        60           1.3647           28.71m
        70           1.3645           21.53m
        80           1.3643           14.35m
        90           1.3641            7.17m
       100           1.3640            0.00s


In [12]:
y_pred = clf.predict(X_test)
y_probs = clf.predict_proba(X_test)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [14]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

56.57792925828424 56.70101275787894 95.99899964333875


In [15]:
bins = np.abs(y_probs[:,1]-0.5)

In [16]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [17]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,451620,854931
1,767198,1340505
2,166471,263306
3,30324,44156
4,2152,2981
5,40,54
6,0,0
7,0,0
8,0,0
9,0,0
